In [1]:
!git clone https://github.com/myshell-ai/OpenVoice
!pip3 install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pydub
!pip3 install faster-whisper whisper_timestamped unidecode eng_to_ipa pypinyin cn2an
!pip3 install -r requirements.txt
!mv OpenVoice/* .
!ls

Cloning into 'OpenVoice'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 273 (delta 72), reused 73 (delta 70), pack-reused 178
Receiving objects: 100% (273/273), 2.40 MiB | 22.09 MiB/s, done.
Resolving deltas: 100% (132/132), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstall

In [2]:
!curl -o x.zip 'https://myshell-public-repo-hosting.s3.amazonaws.com/checkpoints_1226.zip'
!rm -rf checkpoints
!unzip x.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  401M  100  401M    0     0  91.2M      0  0:00:04  0:00:04 --:--:-- 98.1M
Archive:  x.zip
   creating: checkpoints/
   creating: checkpoints/converter/
  inflating: checkpoints/converter/config.json  
  inflating: checkpoints/converter/checkpoint.pth  
   creating: checkpoints/base_speakers/
   creating: checkpoints/base_speakers/ZH/
  inflating: checkpoints/base_speakers/ZH/config.json  
  inflating: checkpoints/base_speakers/ZH/checkpoint.pth  
  inflating: checkpoints/base_speakers/ZH/zh_default_se.pth  
   creating: checkpoints/base_speakers/EN/
  inflating: checkpoints/base_speakers/EN/config.json  
  inflating: checkpoints/base_speakers/EN/en_style_se.pth  
  inflating: checkpoints/base_speakers/EN/en_default_se.pth  
  inflating: checkpoints/base_speakers/EN/checkpoint.pth  


In [3]:
import os
import torch
import se_extractor
from api import BaseSpeakerTTS, ToneColorConverter

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [4]:
!pip install wavmark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.3 MB/s eta 0:00:00


# 测试英文语音克隆

In [5]:
ckpt_base = 'checkpoints/base_speakers/EN'
ckpt_converter = 'checkpoints/converter'
device = 'cuda:0'
output_dir = 'outputs'

base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)BERP_none0.30_mean1.81_std1.81.model.pkl:   0%|          | 0.00/10.0M [00:00<?, ?B/s]

Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


In [6]:
source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)
reference_speaker = 'resources/example_reference.mp3'
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


[(0.0, 14.5666875)]
after vad: dur = 14.566


In [7]:
save_path = f'{output_dir}/english.wav'

# Run the base speaker tts
text = ("We introduce OpenVoice, a versatile instant voice cloning approach that"
        "requires only a short audio clip from the reference speaker to replicate their"
        "voice and generate speech in multiple languages.")
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)


# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message)

 > Text splitted to sentences.
We introduce OpenVoice, a versatile instant voice cloning approach thatrequires only a short audio clip from the reference speaker to replicate theirvoice and generate speech in multiple languages.
 > ===========================
wi ˌɪntɹəˈdus ˈoʊpən vɔɪs, ə ˈvəɹsətəɫ ˈɪnstənt vɔɪs ˈkloʊnɪŋ əˈpɹoʊtʃ thatɹequiɹes* ˈoʊnli ə ʃɔɹt ˈɑdiˌoʊ klɪp fɹəm ðə ˈɹɛfəɹəns ˈspikəɹ tɪ ˈɹɛplɪˌkeɪt theiɹvoice* ənd ˈdʒɛnəɹˌeɪt spitʃ ɪn ˈməltəpəɫ ˈlæŋgwɪdʒɪz.
 length:212
 length:210


In [8]:
import IPython.display as idp
idp.Audio("outputs/english.wav")

# 克隆中文声音

# 新段落

In [9]:
import os
os.system('curl -o seven.mp3 https://host.ddot.cc/tmp_ultrasev.mp3')

0

In [11]:
reference_speaker = 'seven.mp3'
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

[(0.0, 69.01), (71.982, 104.69), (105.998, 114.354), (114.446, 128.53), (129.006, 134.29), (134.478, 143.602), (144.942, 151.602), (155.054, 195.314), (195.342, 204.178), (205.71, 208.786), (209.518, 220.882), (221.646, 229.49), (229.55, 243.09), (243.118, 252.658), (252.75, 276.05), (276.206, 306.162), (311.214, 331.25), (333.23, 356.69), (357.294, 364.594), (364.718, 381.554)]
after vad: dur = 360.5740816326531


In [15]:
ckpt_base = 'checkpoints/base_speakers/ZH'
base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

source_se = torch.load(f'{ckpt_base}/zh_default_se.pth').to(device)
save_path = f'{output_dir}/chinese.wav'

# Run the base speaker tts
text = """我從未親身體驗過金錢的觸感，對財富不抱多大的興趣。因為我覺得最幸福的時刻是當老師，每個月只領取91元薪酬。这些時光充滿了教育的滿足和心靈的豐富，這遠比金錢的追逐更令我快樂。"""
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='Chinese', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path,
    src_se=source_se,
    tgt_se=target_se,
    output_path=save_path,
    message=encode_message)

Loaded checkpoint 'checkpoints/base_speakers/ZH/checkpoint.pth'
missing/unexpected keys: [] []
 > Text splitted to sentences.
我從未親身體驗過金錢的觸感,
對財富不抱多大的興趣.
因為我覺得最幸福的時刻是當老師,
每個月只領取91元薪酬.
这些時光充滿了教育的滿足和心靈的豐富,
這遠比金錢的追逐更令我快樂.
 > ===========================
wo↓↑ tsʰʊŋ↑ weɪ↓ tʃʰin→s`ən→ tʰi↓↑jɛn↓ k⁼wo↓tʃ⁼in→tʃʰjɛn↑ t⁼ə ts`ʰu↓k⁼an↓↑,
 length:75
 length:75
t⁼weɪ↓ tsʰaɪ↑fu↓ p⁼u↓ p⁼ɑʊ↓ t⁼wo→ t⁼a↓ t⁼ə ʃiŋ↓tʃʰɥ↓.
 length:53
 length:53
in→weɪ↓ wo↓↑ tʃ⁼ɥɛ↑t⁼ə↑ ts⁼weɪ↓ ʃiŋ↓fu↑ t⁼ə s`ɹ`↑kʰə↓ s`ɹ`↓ t⁼ɑŋ→lɑʊ↓↑s`ɹ`→,
 length:76
 length:76
meɪ↓↑k⁼ə↓ ɥɛ↓ ts`⁼ɹ`↓↑ liŋ↓↑tʃʰɥ↓↑ tʃ⁼joʊ↓↑s`ɹ`↑i→ ɥæn↑ ʃin→ts`ʰoʊ↑.
 length:68
 length:68
ts`⁼ə↓ʃiɛ→ s`ɹ`↑k⁼uɑŋ→ ts`ʰʊŋ→man↓↑ lə tʃ⁼iɑʊ↓ɥ↓ t⁼ə man↓↑ts⁼u↑ xə↑ ʃin→liŋ↑ t⁼ə fəŋ→fu↓,
 length:89
 length:89
ts`⁼ə↓ɥæn↓↑ p⁼i↓↑tʃ⁼in→tʃʰjɛn↑ t⁼ə ts`⁼weɪ→ts`⁼u↑ k⁼əŋ↓liŋ↓ wo↓↑ kʰwaɪ↓lə↓.
 length:75
 length:75


In [16]:
import os
os.listdir('outputs')

['chinese.wav', 'english.wav', 'tmp.wav']

In [17]:
idp.Audio("outputs/chinese.wav")